In [3]:
import gymnasium as gym
import numpy as np

env = gym.make('FrozenLake-v1',desc=None,map_name='4x4',is_slippery=True)
env.reset()

(0, {'prob': 1})

In [4]:
ns = env.observation_space.n
num_episodes = 10000
discount_rate = 0.9

def MCLearning(env, num_eposides, discout_rate):
    sum_return = np.zeros(ns)
    num_visit_of_states = np.zeros(ns)
    V = np.zeros(ns)
    for episode in range(num_episodes):
        ep_visited = []
        ep_reward  = []
        current_state, prob = env.reset()
        ep_visited.append(current_state)
        while True:
            a = env.action_space.sample()
            next_state, r, done, prob,_ = env.step(a)
            ep_reward.append(r)
            if done:
                break
            else:
                ep_visited.append(next_state)
        
        G=0
        for i in range(len(ep_visited)-1,-1,-1):
            temp_state = ep_visited[i]
            temp_reward = ep_reward[i]
            G = discount_rate*G + temp_reward
            if temp_state not in ep_visited[0:i]:
                num_visit_of_states[temp_state] += 1
                sum_return[temp_state] += G

    for i in range(ns):
        if num_visit_of_states[i] != 0:
            V[i] = sum_return[i]/num_visit_of_states[i]

    return V

In [5]:
def evaluate_policy(env, V, policy, discount_rate, max_iter, tol):
    conv_track = []
    for iteration in range(max_iter):
        conv_track.append(np.linalg.norm(V))
        V_new = np.zeros(ns)
        for state in env.P:
            outer_sum = 0
            for action in env.P[state]:
                inner_sum = 0
                for prob, next_state, reward, done in env.P[state][action]:
                    inner_sum += prob*(reward + discount_rate*V[next_state])
                outer_sum += policy[state][action]*inner_sum
            V_new[state] = outer_sum
        if(np.max(np.abs(V_new-V)) < tol):
            break
        V=V_new
    return V